In [1]:
import pickle as pkl
import numpy as np
import os
import torch

from tqdm import tqdm


os.chdir('/home/jcsanguino10/local_citation_model/recommender-fusion-recsys/loaders')
from create_dataloader_sequential import (load_course_encoder)

os.chdir('/home/jcsanguino10/local_citation_model/recommender-fusion-recsys/architectures/Sequence')
from sec_transformer_pytorch import (create_model, load_pytorch_weights)

os.chdir('/home/jcsanguino10/local_citation_model/recommender-fusion-recsys/architectures/Multimodal')
from multimodal import Autoencoder, MultimodalModel






In [2]:
if torch.cuda.is_available():
    # Change to just a particular GPU changing the enviroment variable
    os.environ["CUDA_VISIBLE_DEVICES"] = "3"
    device = torch.device("cuda")
    # Change to just use a particular GPU via torch
    #torch.cuda.set_device("cuda:3")
    print(torch.cuda.current_device())

0


In [3]:
# Paths from the label encoder dicts 

PATH_TO_LABEL_ENCODER = '/home/jcsanguino10/local_citation_model/data/processed/'

# Path to folder with datasets

PATH_TO_DATASETS = '/home/jcsanguino10/local_citation_model/data/'

# Path to folder with checkpoints best models

PATH_TO_CHECKPOINTS = '/home/jcsanguino10/local_citation_model/models/'

In [4]:
label_encoder, dicts = load_course_encoder('/home', PATH_TO_LABEL_ENCODER)

Loading existing mappings from /home/jcsanguino10/local_citation_model/data/processed


In [5]:
len(label_encoder.classes_)  # List of all course IDs in the dataset

197

In [6]:
import pandas as pd
df_binary = pd.read_pickle(f'{PATH_TO_DATASETS}train_binary_all_vectors_128_01_transe_seqvec.pkl')
df_bpr_df = pd.read_pickle(f'{PATH_TO_DATASETS}train_bpr_all_vectors_128_01_transe_seqvec.pkl')

In [7]:
df_binary.head()

,user_id,item_id,item_seq,item_text_embedding,user_text_embedding,user_cum_text_embedding,item_bpr_embedding,user_bpr_embedding,item_graph_embedding,user_graph_embedding,label,full_item_seq,parsed_item_seq,user_sequence_embedding
0,0,6863,[6863],"[2.2087095e-05, 0.01357965, 0.013899612, -0.03...","[-0.022005824, -0.033391304, -0.013403211, -0....","[2.2087095e-05, 0.01357965, 0.013899612, -0.03...","[-0.06554511, 0.082159385, -0.024771133, -0.08...","[-0.16485311, 0.13305487, -0.109800704, -0.159...","[0.027043026, -0.028548103, -0.005808171, -0.1...","[-0.21868221, 0.107340455, 0.20054282, 0.15184...",1,"[6863, 6864]","[2, 3]","[-3.2515903, -4.4112086, 1.4216669, -2.70002, ..."
1,0,6864,"[6863, 6864]","[-0.044033736, -0.08036226, -0.040706035, -0.0...","[-0.022005824, -0.033391304, -0.013403211, -0....","[-0.022005824, -0.033391304, -0.013403211, -0....","[-0.09407239, 0.08107624, -0.042453628, -0.100...","[-0.16485311, 0.13305487, -0.109800704, -0.159...","[-0.05497769, -0.049202707, -0.13520204, -0.09...","[-0.21868221, 0.107340455, 0.20054282, 0.15184...",1,"[6863, 6864]","[2, 3]","[-3.2515903, -4.4112086, 1.4216669, -2.70002, ..."
2,1,6865,[6865],"[0.06792896, -0.059695832, -0.013778767, 0.005...","[0.034128785, -0.054559205, 0.0015806267, -0.0...","[0.06792896, -0.059695832, -0.013778767, 0.005...","[-0.109637335, 0.08800741, -0.114228204, -0.05...","[-0.15681404, 0.14787957, -0.14133789, -0.1282...","[0.061120644, 0.047605906, 0.1170312, 0.092423...","[0.16515407, 0.18278724, -0.07278056, -0.06131...",1,"[6865, 6866]","[4, 5]","[-0.3152992, -1.0941633, 1.9535718, -1.170749,..."
3,1,6866,"[6865, 6866]","[0.00032860466, -0.049422577, 0.01694002, -0.0...","[0.034128785, -0.054559205, 0.0015806267, -0.0...","[0.034128785, -0.054559205, 0.0015806267, -0.0...","[-0.05689768, 0.085094474, -0.04034279, -0.072...","[-0.15681404, 0.14787957, -0.14133789, -0.1282...","[-0.10074902, 0.19556874, 0.08297272, -0.14672...","[0.16515407, 0.18278724, -0.07278056, -0.06131...",1,"[6865, 6866]","[4, 5]","[-0.3152992, -1.0941633, 1.9535718, -1.170749,..."
4,2,6867,[6867],"[-0.0041599325, -0.058022052, 0.0056412797, -0...","[0.01798853, -0.038172975, -0.0087622935, -0.0...","[-0.0041599325, -0.058022052, 0.0056412797, -0...","[0.07457628, -0.06646576, 0.07522122, 0.040138...","[-0.12063337, 0.13376932, -0.13144864, -0.0975...","[-0.07019904, -0.19594775, -0.1563805, -0.0412...","[-0.21761492, 0.1274874, 0.19495715, -0.179376...",1,"[6867, 6868, 6869, 6870, 6871, 6872, 6873, 687...","[6, 7, 8, 9, 10, 11, 12, 13, 14, 3, 15, 2, 16,...","[-0.97801095, -1.112409, 0.3110269, -0.4820084..."


In [8]:
def concat_columns_to_tensor(df, columns, new_column_name):
    """
    Concatenates specified columns in a DataFrame and creates a tensor.
    The resulting tensor is saved in a new column.

    Args:
        df (pd.DataFrame): The input DataFrame.
        columns (list): List of column names to concatenate.
        new_column_name (str): Name of the new column to store the tensor.

    Returns:
        pd.DataFrame: The DataFrame with the new column containing tensors.
    """
    df[new_column_name] = df[columns].apply(
        lambda row: torch.tensor([item for col in columns for item in row[col]], dtype=torch.float),
        axis=1
    )
    return df

In [9]:
df_binary = concat_columns_to_tensor(df_binary, ['item_text_embedding', 'item_bpr_embedding', 'item_graph_embedding'], 'course_full_embeddings')
df_binary = concat_columns_to_tensor(df_binary, ['user_text_embedding', 'user_bpr_embedding', 'user_graph_embedding', 'user_sequence_embedding'], 'user_full_embeddings')

In [10]:
df_bpr_df = concat_columns_to_tensor(df_bpr_df, ['pos_item_text_embedding', 'pos_item_bpr_embedding', 'pos_item_graph_embedding'], 'pos_course_full_embeddings')
df_bpr_df = concat_columns_to_tensor(df_bpr_df, ['neg_item_text_embedding', 'neg_item_bpr_embedding', 'neg_item_graph_embedding'], 'neg_course_full_embeddings')
df_bpr_df = concat_columns_to_tensor(df_bpr_df, ['user_text_embedding', 'user_bpr_embedding', 'user_graph_embedding', 'user_sequence_embedding'], 'user_full_embeddings')

# Autoencoder training 

In [11]:
def train_autoencoder_and_extract_encoder(data, input_dim, encoding_dims, epochs=50, lr=1e-3, 
                                         save_path=None, device='cuda', verbose=True):
    """
    Train an autoencoder.
    
    Args:
        data (torch.Tensor): Training data tensor of shape (batch_size, input_dim)
        input_dim (int): Dimension of input features
        encoding_dims (list): List of hidden layer dimensions for encoder
                             Example: [512, 256, 128] for 3-layer encoder
        epochs (int): Number of training epochs
        lr (float): Learning rate
        save_path (str): Path to save the best autoencoder model (optional)
        device (str): Device to train on ('cpu' or 'cuda')
        verbose (bool): Whether to print training progress
        
    Returns:
        encoder (nn.Module): The trained autoencoder model
    """
    
    if verbose:
        print(f"🚀 Starting autoencoder training...")
        print(f"Input dimension: {input_dim}")
        print(f"Encoding dimensions: {encoding_dims}")
        print(f"Final encoding dimension: {encoding_dims[-1]}")
        print(f"Training data shape: {data.shape}")
    
    # Create autoencoder
    autoencoder = Autoencoder(input_dim=input_dim, encoding_dims=encoding_dims)
    
    if verbose:
        print(f"📊 Autoencoder architecture created")
        print(f"Encoder layers: {len(autoencoder.encoder)}")
        print(f"Decoder layers: {len(autoencoder.decoder)}")
    
    # Train the autoencoder using the enhanced train_autoencoder method
    trained_autoencoder = Autoencoder.train_autoencoder(
        autoencoder=autoencoder,
        data=data,
        epochs=epochs,
        lr=lr,
        save_path=save_path,
        device=device
    )
    
    
    if verbose:
        print(f"✅ Training completed!")
        print(f"🔧 Encoder extracted successfully")
        print(f"📐 Encoder output dimension: {encoding_dims[-1]}")
    
    return trained_autoencoder

In [12]:
course_tensor = [torch.tensor(x, dtype=torch.float32) for x in df_binary['course_full_embeddings'].values]
embeddings__course_tensor = torch.stack(course_tensor)

user_tensor = [torch.tensor(x, dtype=torch.float32) for x in df_binary['user_full_embeddings'].values]
embeddings_user_tensor = torch.stack(user_tensor)

/tmp/ipykernel_2986274/1407629466.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  course_tensor = [torch.tensor(x, dtype=torch.float32) for x in df_binary['course_full_embeddings'].values]
/tmp/ipykernel_2986274/1407629466.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  user_tensor = [torch.tensor(x, dtype=torch.float32) for x in df_binary['user_full_embeddings'].values]


In [13]:
course_encoder = train_autoencoder_and_extract_encoder(embeddings__course_tensor, embeddings__course_tensor.shape[1], [680, 560, 360], save_path=f'{PATH_TO_CHECKPOINTS}encoder_course.pth' ,epochs=100, lr=1e-3, verbose=False)

Data split: 31456 training samples, 7864 validation samples
Training autoencoder for 100 epochs...
✅ New best model saved at epoch 1 with val_loss: 0.002894
Epoch 1/100, Train Loss: 0.003350, Val Loss: 0.002894, Best Val Loss: 0.002894
✅ New best model saved at epoch 2 with val_loss: 0.002669
✅ New best model saved at epoch 3 with val_loss: 0.002537
✅ New best model saved at epoch 4 with val_loss: 0.002485
✅ New best model saved at epoch 5 with val_loss: 0.002439
Epoch 5/100, Train Loss: 0.002535, Val Loss: 0.002439, Best Val Loss: 0.002439
✅ New best model saved at epoch 6 with val_loss: 0.002404
✅ New best model saved at epoch 7 with val_loss: 0.002377
✅ New best model saved at epoch 8 with val_loss: 0.002340
✅ New best model saved at epoch 9 with val_loss: 0.002289
✅ New best model saved at epoch 10 with val_loss: 0.002240
Epoch 10/100, Train Loss: 0.002324, Val Loss: 0.002240, Best Val Loss: 0.002240
✅ New best model saved at epoch 11 with val_loss: 0.002192
✅ New best model saved 

In [14]:
user_encoder = train_autoencoder_and_extract_encoder(embeddings_user_tensor, embeddings_user_tensor.shape[1], [680, 560, 360], save_path=f'{PATH_TO_CHECKPOINTS}encoder_user.pth' ,epochs=100, lr=1e-3, verbose=False)

Data split: 31456 training samples, 7864 validation samples
Training autoencoder for 100 epochs...
✅ New best model saved at epoch 1 with val_loss: 0.266960
Epoch 1/100, Train Loss: 0.273164, Val Loss: 0.266960, Best Val Loss: 0.266960
✅ New best model saved at epoch 2 with val_loss: 0.249679
✅ New best model saved at epoch 3 with val_loss: 0.206341
✅ New best model saved at epoch 5 with val_loss: 0.152368
Epoch 5/100, Train Loss: 0.267226, Val Loss: 0.152368, Best Val Loss: 0.152368
Epoch 10/100, Train Loss: 0.184694, Val Loss: 0.167825, Best Val Loss: 0.152368
✅ New best model saved at epoch 11 with val_loss: 0.150768
✅ New best model saved at epoch 12 with val_loss: 0.144200
✅ New best model saved at epoch 13 with val_loss: 0.141376
✅ New best model saved at epoch 14 with val_loss: 0.127704
✅ New best model saved at epoch 15 with val_loss: 0.121834
Epoch 15/100, Train Loss: 0.138565, Val Loss: 0.121834, Best Val Loss: 0.121834
✅ New best model saved at epoch 19 with val_loss: 0.1185

# Multimodal training

In [15]:
modality_dims = {
    'course': embeddings__course_tensor.shape[1],
    'user': embeddings_user_tensor.shape[1]
}

In [16]:
model_encoder = MultimodalModel(modality_dims, use_bpr=True, fusion_method='by_autoencoder',shared_dim=32, layers_per_modality=[256 ,128, 64] ,autoencoders={'course': course_encoder, 'user': user_encoder}, autoencoder_output_dim=360)

User feature layer input dim: 360
Course feature layer input dim: 360


In [17]:
# Create a dataloader for BPR training using the df_bpr_df dataframe with the columns: pos_course_full_embeddings, neg_course_full_embeddings, user_full_embeddings
class BPRDataset(torch.utils.data.Dataset):
    def __init__(self, df):
        self.df = df

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        user_feat = self.df.iloc[idx]['user_full_embeddings']
        pos_course_feat = self.df.iloc[idx]['pos_course_full_embeddings']
        neg_course_feat = self.df.iloc[idx]['neg_course_full_embeddings']
        return {
            'user': torch.tensor(user_feat, dtype=torch.float),
            'course_positive': torch.tensor(pos_course_feat, dtype=torch.float),
            'course_negative': torch.tensor(neg_course_feat, dtype=torch.float)
        }
bpr_dataset = BPRDataset(df_bpr_df)
bpr_dataloader = torch.utils.data.DataLoader(bpr_dataset, batch_size=64, shuffle=True)

In [18]:
model_encoder.train_model(
    train_loader=bpr_dataloader,
    epochs=20,
    lr=1e-3,
    device='cuda',
    save_path=f'{PATH_TO_CHECKPOINTS}multimodal_encoder_bpr_model.pth'
)

Training multimodal model for 20 epochs...
No validation dataset provided - using training loss for model selection


/tmp/ipykernel_2986274/242066864.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'user': torch.tensor(user_feat, dtype=torch.float),
/tmp/ipykernel_2986274/242066864.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'course_positive': torch.tensor(pos_course_feat, dtype=torch.float),
/tmp/ipykernel_2986274/242066864.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'course_negative': torch.tensor(neg_course_feat, dtype=torch.float)


✅ New best model saved at epoch 1 with val_loss: 0.511510
Epoch 1/20, Train Loss: 0.511510, Best Loss: 0.511510
✅ New best model saved at epoch 2 with val_loss: 0.406950
Epoch 2/20, Train Loss: 0.406950, Best Loss: 0.406950
✅ New best model saved at epoch 3 with val_loss: 0.358461
Epoch 3/20, Train Loss: 0.358461, Best Loss: 0.358461
✅ New best model saved at epoch 4 with val_loss: 0.336161
Epoch 4/20, Train Loss: 0.336161, Best Loss: 0.336161
✅ New best model saved at epoch 5 with val_loss: 0.322812
Epoch 5/20, Train Loss: 0.322812, Best Loss: 0.322812
✅ New best model saved at epoch 6 with val_loss: 0.318244
Epoch 6/20, Train Loss: 0.318244, Best Loss: 0.318244
✅ New best model saved at epoch 7 with val_loss: 0.312302
Epoch 7/20, Train Loss: 0.312302, Best Loss: 0.312302
✅ New best model saved at epoch 8 with val_loss: 0.305751
Epoch 8/20, Train Loss: 0.305751, Best Loss: 0.305751
✅ New best model saved at epoch 9 with val_loss: 0.300331
Epoch 9/20, Train Loss: 0.300331, Best Loss: 0

In [19]:
model_no_encoder = MultimodalModel(modality_dims, use_bpr=True, fusion_method='concat',shared_dim=32, layers_per_modality=[680, 560, 360 ,256 ,128, 64] ,autoencoders=None, autoencoder_output_dim=None)

User feature layer input dim: 1158
Course feature layer input dim: 960


In [20]:
model_no_encoder.train_model(
    train_loader=bpr_dataloader,
    epochs=20,
    lr=1e-3,
    device='cuda',
    save_path=f'{PATH_TO_CHECKPOINTS}multimodal_no_encoder_bpr_model.pth'
)

Training multimodal model for 20 epochs...
No validation dataset provided - using training loss for model selection


/tmp/ipykernel_2986274/242066864.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'user': torch.tensor(user_feat, dtype=torch.float),
/tmp/ipykernel_2986274/242066864.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'course_positive': torch.tensor(pos_course_feat, dtype=torch.float),
/tmp/ipykernel_2986274/242066864.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'course_negative': torch.tensor(neg_course_feat, dtype=torch.float)


✅ New best model saved at epoch 1 with val_loss: 0.522595
Epoch 1/20, Train Loss: 0.522595, Best Loss: 0.522595
✅ New best model saved at epoch 2 with val_loss: 0.435717
Epoch 2/20, Train Loss: 0.435717, Best Loss: 0.435717
✅ New best model saved at epoch 3 with val_loss: 0.414116
Epoch 3/20, Train Loss: 0.414116, Best Loss: 0.414116
✅ New best model saved at epoch 4 with val_loss: 0.345178
Epoch 4/20, Train Loss: 0.345178, Best Loss: 0.345178
✅ New best model saved at epoch 5 with val_loss: 0.287274
Epoch 5/20, Train Loss: 0.287274, Best Loss: 0.287274
✅ New best model saved at epoch 6 with val_loss: 0.252212
Epoch 6/20, Train Loss: 0.252212, Best Loss: 0.252212
✅ New best model saved at epoch 7 with val_loss: 0.215344
Epoch 7/20, Train Loss: 0.215344, Best Loss: 0.215344
✅ New best model saved at epoch 8 with val_loss: 0.176293
Epoch 8/20, Train Loss: 0.176293, Best Loss: 0.176293
✅ New best model saved at epoch 9 with val_loss: 0.162344
Epoch 9/20, Train Loss: 0.162344, Best Loss: 0

In [21]:
## Create dataloaders for binary training using the df_binary dataframe with the columns: course_full_embeddings, user_full_embeddings
class BinaryDataset(torch.utils.data.Dataset):
    def __init__(self, df):
        self.df = df

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        user_feat = self.df.iloc[idx]['user_full_embeddings']
        course_feat = self.df.iloc[idx]['course_full_embeddings']
        label = self.df.iloc[idx]['label']
        return {
            'user': torch.tensor(user_feat, dtype=torch.float),
            'course_positive': torch.tensor(course_feat, dtype=torch.float),
            'targets': torch.tensor(label, dtype=torch.float)
        }

In [22]:
## Split df_binary into train and validation sets
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(df_binary, test_size=0.2, random_state=42)

binary_dataset = BinaryDataset(train_df)
binary_dataloader = torch.utils.data.DataLoader(binary_dataset, batch_size=64, shuffle=True)

val_dataset = BinaryDataset(val_df)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=64, shuffle=False)

In [23]:
model_binary = MultimodalModel(modality_dims, use_bpr=False, fusion_method='by_autoencoder',shared_dim=32, layers_per_modality=[256 ,128, 64] ,autoencoders={'course': course_encoder, 'user': user_encoder}, autoencoder_output_dim=360)

User feature layer input dim: 360
Course feature layer input dim: 360


In [24]:
model_binary_no_encoder = MultimodalModel(modality_dims, use_bpr=False, fusion_method='concat',shared_dim=32, layers_per_modality=[680, 560, 360 ,256 ,128, 64] ,autoencoders=None, autoencoder_output_dim=None)

User feature layer input dim: 1158
Course feature layer input dim: 960


In [25]:
model_binary.train_model(
    train_loader=binary_dataloader,
    val_loader=val_dataloader,
    epochs=20,
    lr=1e-3,
    device='cuda',
    save_path=f'{PATH_TO_CHECKPOINTS}multimodal_encoder_binary_model.pth'
)

Training multimodal model for 20 epochs...
Using validation dataset with 123 batches


/tmp/ipykernel_2986274/754840888.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'user': torch.tensor(user_feat, dtype=torch.float),
/tmp/ipykernel_2986274/754840888.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'course_positive': torch.tensor(course_feat, dtype=torch.float),


✅ New best model saved at epoch 1 with val_loss: 0.461161
Epoch 1/20, Train Loss: 0.481609, Val Loss: 0.461161, Best Val Loss: 0.461161
Epoch 2/20, Train Loss: 0.456511, Val Loss: 0.463141, Best Val Loss: 0.461161
Epoch 3/20, Train Loss: 0.450777, Val Loss: 0.468032, Best Val Loss: 0.461161
✅ New best model saved at epoch 4 with val_loss: 0.455969
Epoch 4/20, Train Loss: 0.449362, Val Loss: 0.455969, Best Val Loss: 0.455969
✅ New best model saved at epoch 5 with val_loss: 0.451952
Epoch 5/20, Train Loss: 0.448433, Val Loss: 0.451952, Best Val Loss: 0.451952
✅ New best model saved at epoch 6 with val_loss: 0.448255
Epoch 6/20, Train Loss: 0.445316, Val Loss: 0.448255, Best Val Loss: 0.448255
✅ New best model saved at epoch 7 with val_loss: 0.444425
Epoch 7/20, Train Loss: 0.442667, Val Loss: 0.444425, Best Val Loss: 0.444425
Epoch 8/20, Train Loss: 0.440303, Val Loss: 0.456076, Best Val Loss: 0.444425
✅ New best model saved at epoch 9 with val_loss: 0.443938
Epoch 9/20, Train Loss: 0.43

In [26]:
model_binary_no_encoder.train_model(
    train_loader=binary_dataloader,
    val_loader=val_dataloader,
    epochs=20,
    lr=1e-3,
    device='cuda',
    save_path=f'{PATH_TO_CHECKPOINTS}multimodal_no_encoder_binary_model.pth'
)

Training multimodal model for 20 epochs...
Using validation dataset with 123 batches


/tmp/ipykernel_2986274/754840888.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'user': torch.tensor(user_feat, dtype=torch.float),
/tmp/ipykernel_2986274/754840888.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'course_positive': torch.tensor(course_feat, dtype=torch.float),


✅ New best model saved at epoch 1 with val_loss: 0.522323
Epoch 1/20, Train Loss: 0.536095, Val Loss: 0.522323, Best Val Loss: 0.522323
✅ New best model saved at epoch 2 with val_loss: 0.406445
Epoch 2/20, Train Loss: 0.453521, Val Loss: 0.406445, Best Val Loss: 0.406445
✅ New best model saved at epoch 3 with val_loss: 0.375662
Epoch 3/20, Train Loss: 0.394084, Val Loss: 0.375662, Best Val Loss: 0.375662
✅ New best model saved at epoch 4 with val_loss: 0.374218
Epoch 4/20, Train Loss: 0.379946, Val Loss: 0.374218, Best Val Loss: 0.374218
Epoch 5/20, Train Loss: 0.372543, Val Loss: 0.393146, Best Val Loss: 0.374218
✅ New best model saved at epoch 6 with val_loss: 0.369182
Epoch 6/20, Train Loss: 0.365980, Val Loss: 0.369182, Best Val Loss: 0.369182
✅ New best model saved at epoch 7 with val_loss: 0.367399
Epoch 7/20, Train Loss: 0.362869, Val Loss: 0.367399, Best Val Loss: 0.367399
✅ New best model saved at epoch 8 with val_loss: 0.355549
Epoch 8/20, Train Loss: 0.358092, Val Loss: 0.35

In [27]:
import pandas as pd
df_test_binary = pd.read_pickle(f'{PATH_TO_DATASETS}test_binary_all_vectors_128_01_transe_seqvec.pkl')
df_test_bpr = pd.read_pickle(f'{PATH_TO_DATASETS}test_bpr_all_vectors_128_01_transe_seqvec.pkl')

In [28]:
df_bpr_df.head()

,user_id,pos_item_id,neg_item_id,item_seq,pos_item_text_embedding,neg_item_text_embedding,user_text_embedding,user_cum_text_embedding,pos_item_bpr_embedding,neg_item_bpr_embedding,user_bpr_embedding,pos_item_graph_embedding,neg_item_graph_embedding,user_graph_embedding,full_item_seq,parsed_item_seq,user_sequence_embedding,pos_course_full_embeddings,neg_course_full_embeddings,user_full_embeddings
0,0,6863,7029,[6863],"[2.2087095e-05, 0.01357965, 0.013899612, -0.03...","[0.005838169, 0.0019245448, 0.013981204, 0.049...","[-0.022005824, -0.033391304, -0.013403211, -0....","[2.2087095e-05, 0.01357965, 0.013899612, -0.03...","[-0.06554511, 0.082159385, -0.024771133, -0.08...","[0.12648226, -0.08695751, 0.1032631, 0.1059599...","[-0.16485311, 0.13305487, -0.109800704, -0.159...","[0.027043026, -0.028548103, -0.005808171, -0.1...","[-0.14888977, 0.050079968, 0.12822092, 0.00475...","[-0.21868221, 0.107340455, 0.20054282, 0.15184...","[6863, 6864]","[2, 3]","[-3.2515903, -4.4112086, 1.4216669, -2.70002, ...","[tensor(2.2087e-05), tensor(0.0136), tensor(0....","[tensor(0.0058), tensor(0.0019), tensor(0.0140...","[tensor(-0.0220), tensor(-0.0334), tensor(-0.0..."
1,0,6864,7029,"[6863, 6864]","[-0.044033736, -0.08036226, -0.040706035, -0.0...","[0.005838169, 0.0019245448, 0.013981204, 0.049...","[-0.022005824, -0.033391304, -0.013403211, -0....","[-0.022005824, -0.033391304, -0.013403211, -0....","[-0.09407239, 0.08107624, -0.042453628, -0.100...","[0.12648226, -0.08695751, 0.1032631, 0.1059599...","[-0.16485311, 0.13305487, -0.109800704, -0.159...","[-0.05497769, -0.049202707, -0.13520204, -0.09...","[-0.14888977, 0.050079968, 0.12822092, 0.00475...","[-0.21868221, 0.107340455, 0.20054282, 0.15184...","[6863, 6864]","[2, 3]","[-3.2515903, -4.4112086, 1.4216669, -2.70002, ...","[tensor(-0.0440), tensor(-0.0804), tensor(-0.0...","[tensor(0.0058), tensor(0.0019), tensor(0.0140...","[tensor(-0.0220), tensor(-0.0334), tensor(-0.0..."
2,1,6865,6923,[6865],"[0.06792896, -0.059695832, -0.013778767, 0.005...","[0.050885703, 0.016394274, -0.0133330505, -0.0...","[0.034128785, -0.054559205, 0.0015806267, -0.0...","[0.06792896, -0.059695832, -0.013778767, 0.005...","[-0.109637335, 0.08800741, -0.114228204, -0.05...","[0.10082436, -0.09402634, 0.100806765, 0.09215...","[-0.15681404, 0.14787957, -0.14133789, -0.1282...","[0.061120644, 0.047605906, 0.1170312, 0.092423...","[0.12331399, 0.03431532, 0.21699251, 0.1532834...","[0.16515407, 0.18278724, -0.07278056, -0.06131...","[6865, 6866]","[4, 5]","[-0.3152992, -1.0941633, 1.9535718, -1.170749,...","[tensor(0.0679), tensor(-0.0597), tensor(-0.01...","[tensor(0.0509), tensor(0.0164), tensor(-0.013...","[tensor(0.0341), tensor(-0.0546), tensor(0.001..."
3,1,6866,6928,"[6865, 6866]","[0.00032860466, -0.049422577, 0.01694002, -0.0...","[0.05351212, -0.040352862, -0.01637018, 0.0096...","[0.034128785, -0.054559205, 0.0015806267, -0.0...","[0.034128785, -0.054559205, 0.0015806267, -0.0...","[-0.05689768, 0.085094474, -0.04034279, -0.072...","[0.1240448, -0.09917505, 0.098442316, 0.094072...","[-0.15681404, 0.14787957, -0.14133789, -0.1282...","[-0.10074902, 0.19556874, 0.08297272, -0.14672...","[-0.0033504472, 0.13650173, -0.19897188, 0.152...","[0.16515407, 0.18278724, -0.07278056, -0.06131...","[6865, 6866]","[4, 5]","[-0.3152992, -1.0941633, 1.9535718, -1.170749,...","[tensor(0.0003), tensor(-0.0494), tensor(0.016...","[tensor(0.0535), tensor(-0.0404), tensor(-0.01...","[tensor(0.0341), tensor(-0.0546), tensor(0.001..."
4,2,6867,7047,[6867],"[-0.0041599325, -0.058022052, 0.0056412797, -0...","[0.036724288, -0.022930954, 0.012853282, -0.01...","[0.01798853, -0.038172975, -0.0087622935, -0.0...","[-0.0041599325, -0.058022052, 0.0056412797, -0...","[0.07457628, -0.06646576, 0.07522122, 0.040138...","[0.13138588, -0.10216947, 0.10394299, 0.092404...","[-0.12063337, 0.13376932, -0.13144864, -0.0975...","[-0.07019904, -0.19594775, -0.1563805, -0.0412...","[0.17877811, 0.13246465, 0.01112991, 0.1806746...","[-0

In [29]:
# Create a mapping of user_id to user_sequence_embedding from the train dataset
user_sequence_mapping = df_binary.drop_duplicates(subset="user_id").set_index('user_id')['user_full_embeddings'].to_dict()

# Replace the user_sequence_embedding in the test dataset using the mapping
df_test_binary['user_full_embeddings'] = df_test_binary['user_id'].map(user_sequence_mapping)
df_test_bpr['user_full_embeddings'] = df_test_bpr['user_id'].map(user_sequence_mapping)

In [30]:
course_sequence_mapping = df_bpr_df.drop_duplicates(subset="pos_item_id").set_index('pos_item_id')['pos_course_full_embeddings'].to_dict()

In [31]:
os.chdir('/home/jcsanguino10/local_citation_model/Secuencial SR')
from evaluation_metrics import calculate_average_mrr, calculate_average_precision_at_k, calculate_average_ndcg_at_k, calculate_average_custom_precision_at_k


In [32]:
def generate_recommendations_per_user(df, model, courses_dict, k=5, batch_size=64):

    user_tensors = [torch.tensor(x) if not isinstance(x, torch.Tensor) else x 
                for x in df["user_full_embeddings"].values]

    user_tensors = torch.stack(user_tensors)

    all_user_embs = user_tensors  # shape [num_users, dim]
    recommendations = []

    for i in tqdm(range(0, len(all_user_embs), batch_size), desc="Generating recommendations"):
        batch = all_user_embs[i:i+batch_size]
        batch_recs = model.generate_k_recommendations(courses_dict, batch, k=k)
        recommendations.extend(batch_recs)

    df["recommendations"] = recommendations
    return df


In [33]:
def test_model(df, models, courses_dict, k):
    for model in models:
        temp_df = generate_recommendations_per_user(df.drop_duplicates(subset="user_id"), model, courses_dict)
        k=5
        courses_test_dataset = temp_df["full_item_seq"].to_list()
        courses_recommended_list = temp_df["recommendations"].to_list()

        avg_mrr = calculate_average_mrr(courses_test_dataset, courses_recommended_list)
        avg_ndcg_at_k = calculate_average_ndcg_at_k(courses_test_dataset, courses_recommended_list, k)
        avg_precision_at_k = calculate_average_precision_at_k(courses_test_dataset, courses_recommended_list, k)
        avg_custom_precision_at_k = calculate_average_custom_precision_at_k(courses_test_dataset, courses_recommended_list, k)
        print(f"Average MRR: {avg_mrr}")
        print(f"Average NDCG@{k}: {avg_ndcg_at_k}")
        print(f"Average Precision@{k}: {avg_precision_at_k}")
        print(f"Average Custom Precision@{k}: {avg_custom_precision_at_k}") 


In [34]:
test_model(df_test_binary.drop_duplicates(subset="user_id"), models=[model_encoder, model_no_encoder, model_binary, model_binary_no_encoder], courses_dict=course_sequence_mapping, k=5)

Generating recommendations:   0%|          | 0/108 [00:00<?, ?it/s]

Generating recommendations: 100%|██████████| 108/108 [00:06<00:00, 17.59it/s]


Average MRR: 0.12431152557190916
Average NDCG@5: 0.17511222410590072
Average Precision@5: 0.06825003642721604
Average Custom Precision@5: 0.20988634707853718


Generating recommendations: 100%|██████████| 108/108 [00:06<00:00, 16.90it/s]


Average MRR: 0.1178007674000692
Average NDCG@5: 0.15994722831188687
Average Precision@5: 0.059565787556460324
Average Custom Precision@5: 0.18412987517606497


Generating recommendations: 100%|██████████| 108/108 [00:07<00:00, 15.41it/s]


Average MRR: 0.13320462382825968
Average NDCG@5: 0.18586588563020812
Average Precision@5: 0.07221331779105097
Average Custom Precision@5: 0.22111564427606975


Generating recommendations: 100%|██████████| 108/108 [00:07<00:00, 14.86it/s]

Average MRR: 0.13147797367526512
Average NDCG@5: 0.1746645209448283
Average Precision@5: 0.06192627130992068
Average Custom Precision@5: 0.19377580261304597
